# Variables

## Import des librairies

In [ ]:
import os
import sys
from rdflib import Graph, Namespace, URIRef, Literal, BNode, XSD

## Définition des variables globales

In [ ]:
# Fichiers existants
ont_file_name = "ontology.ttl" # Fusion des deux fichiers précédents
ruleset_file_name = "rules.pie"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"
facts_ttl_file_name = "facts_data.ttl"
implicit_to_facts_ttl_file_name = "implicit_to_facts.ttl"

# Dossiers existants
data_folder_name = "data"
mapping_folder_name = "mappings"

# Dossiers créés durant le traitement
tmp_folder_name = "tmp_files"

# Nom du répertoire GraphDB pour les faits
facts_repository_name = "addresses_from_factoids_2"

# Définition des graphes nommés
ontology_named_graph_name = "ontology"
facts_named_graph_name = "facts"
factoids_named_graph_name = "factoids"
time_named_graph_name = "time"
permanent_named_graph_name = "permanent"
tmp_named_graph_name = "temporary"

# Définition de namespaces et de préfixes associés
namespace_prefixes = {"addr":Namespace("http://rdf.geohistoricaldata.org/def/address#"),
                      "atype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/attributeType/"),
                      "ltype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/"),
                      "lrtype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/"),
                      "ctype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/changeType/"),
                      "facts":Namespace("http://rdf.geohistoricaldata.org/id/address/facts/"),
                      "factoids":Namespace("http://rdf.geohistoricaldata.org/id/address/factoids/"),
                      "geo": Namespace("http://www.opengis.net/ont/geosparql#"),
                      "skos": Namespace("http://www.w3.org/2004/02/skos/core#"),
                      "rdf": Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#"),
                      "rdfs": Namespace("http://www.w3.org/2000/01/rdf-schema#"),
                      }

# URIs pour accéder aux logiciels GraphDB et Ontotext-Refine
graphdb_url = "http://localhost:7200"
ontorefine_url = "http://localhost:7333"

# Commande pour lancer `Ontorefine`, dépend de l'OS et d'où il se situe
# ontorefine_cmd = "ontorefine-cli" # Nom sans chemin
ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli" #Ubuntu
# ontorefine_cmd = "/Applications/Ontotext\\ Refine.app/Contents/app/bin/ontorefine-cli" #MacOS

py_code_folder_path = "../code"

## Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)
facts_ttl_file = os.path.join(tmp_folder, facts_ttl_file_name)
implicit_to_facts_ttl_file = os.path.join(tmp_folder, implicit_to_facts_ttl_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import wikidata as wd
import ontorefine as otr
import graphdb as gd
import graphrdf as gr
import strprocessing as sp
import geomprocessing as gp
import multisourcesprocessing as msp
import factoidscreation as fc
import timemanagement as tm

## Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait.

In [ ]:
# Il se peut que la suppression d'un répertoire ne fonctionne pas donc pour éviter la suppresion au moment de la réinitialisation (suppression + (re)création)
# `allow_removal` doit valoir False et dans ce cas-là, le répertoire sera juste vidé.
allow_removal = False
disable_same_as = False

# gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, ruleset_file="owl2-rl-optimized", disable_same_as=disable_same_as, allow_removal=allow_removal)
gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, ruleset_file=ruleset_file, disable_same_as=disable_same_as, allow_removal=allow_removal)

## Import des ontologies

In [ ]:
gd.load_ontologies(graphdb_url, facts_repository_name, [ont_file], ontology_named_graph_name)

## Définition de variables liées aux sources

### Voies de Paris via Wikidata

* `wd` pour "wikidata"
* `wdpt` pour "wikidata paris thoroughfares"
* `wdpa` pour "wikidata paris areas"
* `wdpl` pour "wikidata paris thoroughfare locations"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de Wikidata
wd_repository_name = "factoids_wikidata"

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpt_csv_file_name = "wd_paris_thoroughfares.csv"
wdpt_csv_file = os.path.join(data_folder, wdpt_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpt_mapping_file_name = "mapping_wikidata_landmarks_thoroughfares.json"
wdpt_mapping_file = os.path.join(mapping_folder, wdpt_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
wdpt_kg_file_name = "wd_paris_thoroughfares.ttl"
wdpt_kg_file = os.path.join(tmp_folder, wdpt_kg_file_name)

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpa_csv_file_name = "wd_paris_areas.csv"
wdpa_csv_file = os.path.join(data_folder, wdpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpa_mapping_file_name = "mapping_wikidata_landmarks_areas.json"
wdpa_mapping_file = os.path.join(mapping_folder, wdpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des zones de Paris
wdpa_kg_file_name = "wd_paris_areas.ttl"
wdpa_kg_file = os.path.join(tmp_folder, wdpa_kg_file_name)

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpl_csv_file_name = "wd_paris_locations.csv"
wdpl_csv_file = os.path.join(data_folder, wdpl_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpl_mapping_file_name = "mapping_wikidata_landmark_relations.json"
wdpl_mapping_file = os.path.join(mapping_folder, wdpl_mapping_file_name)

# Fichier TTL pour structurer les connaissances des zones de Paris
wdpl_kg_file_name = "wd_paris_thoroughfare_locations.ttl"
wdpl_kg_file = os.path.join(tmp_folder, wdpl_kg_file_name)

# Fichier TTL final des factoïdes de Wikidata
wd_factoids_kg_file_name = "wd_factoids.ttl"
wd_factoids_kg_file = os.path.join(tmp_folder, wd_factoids_kg_file_name)
wd_permanent_kg_file_name = "wd_permanent.ttl"
wd_permanent_kg_file = os.path.join(tmp_folder, wd_permanent_kg_file_name)

### Nomenclature des voies de la ville de Paris

Les données de la ville de Paris sont composées de deux jeux :
* [dénominations des emprises des voies actuelles](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [dénominations caduques des voies](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Les voies actuelles ont une emprise géométrique contrairement aux anciennes voies.

* `vpt` pour "ville paris thoroughfares"
* `vpta` pour "ville paris thoroughfares actuelles"
* `vptc` pour "ville paris thoroughfares caduques"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de la BAN
vpt_repository_name = "factoids_ville_de_paris"

# Fichier CSV pour stocker le résultat de la requête de la sélection
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(data_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(data_folder, vptc_csv_file_name)

# Propriétés liées au fichier `vpta_csv_file` (colonne décrivant la géométrie, séparateur de colonnes...)
vpta_csv_file_delimiter = ";"
vpta_csv_file_geom_col_name = "geo_shape"

# Le fichier `vpta_csv_file_name` sera modifié durant le traitement. Le fichier ci-dessous sera la version modifiée.
vpta_mod_csv_file_name = "denominations-emprises-voies-actuelles_wkt.csv"
vpta_mod_csv_file = os.path.join(tmp_folder, vpta_mod_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
vpta_mapping_file_name = "mapping_voies_paris_actuelles.json"
vpta_mapping_file = os.path.join(mapping_folder, vpta_mapping_file_name)
vptc_mapping_file_name = "mapping_voies_paris_caduques.json"
vptc_mapping_file = os.path.join(mapping_folder, vptc_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
vpta_kg_file_name = "voies_paris_actuelles.ttl"
vpta_kg_file = os.path.join(tmp_folder, vpta_kg_file_name)
vptc_kg_file_name = "voies_paris_caduques.ttl"
vptc_kg_file = os.path.join(tmp_folder, vptc_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
vpt_factoids_kg_file_name = "vpt_factoids.ttl"
vpt_factoids_kg_file = os.path.join(tmp_folder, vpt_factoids_kg_file_name)
vpt_permanent_kg_file_name = "vpt_permanent.ttl"
vpt_permanent_kg_file = os.path.join(tmp_folder, vpt_permanent_kg_file_name)

vpta_time_description = {
    "start_time" : {"stamp":"2024-02-10T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### Base Adresse Nationale (BAN)

Données de la [Base Adresse Nationale (BAN)](https://adresse.data.gouv.fr/base-adresse-nationale), accessible [ici](https://adresse.data.gouv.fr/data/ban/adresses/latest/csv)

`bpa` pour "BAN paris addresses"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de la BAN
bpa_repository_name = "factoids_ban"

# Fichier CSV pour stocker le résultat de la requête de la sélection
bpa_csv_file_name = "ban_adresses.csv"
bpa_csv_file = os.path.join(data_folder, bpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
bpa_mapping_file_name = "mapping_ban_adresses.json"
bpa_mapping_file = os.path.join(mapping_folder, bpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
bpa_kg_file_name = "ban_adresses.ttl"
bpa_kg_file = os.path.join(tmp_folder, bpa_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
bpa_factoids_kg_file_name = "ban_factoids.ttl"
bpa_factoids_kg_file = os.path.join(tmp_folder, bpa_factoids_kg_file_name)
bpa_permanent_kg_file_name = "ban_permanent.ttl"
bpa_permanent_kg_file = os.path.join(tmp_folder, bpa_permanent_kg_file_name)

bpa_time_description = {
    "start_time" : {"stamp":"2024-01-01T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### OpenStreetMap (OSM)

Extraction des données d'OpenStreetMap

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données d'OSM
osm_repository_name = "factoids_osm"

# Fichiers CSV pour stocker les résultats des requêtes de la sélection
osm_csv_file_name = "osm_adresses.csv"
osm_csv_file = os.path.join(data_folder, osm_csv_file_name)
osm_hn_csv_file_name = "osm_hn_adresses.csv"
osm_hn_csv_file = os.path.join(data_folder, osm_hn_csv_file_name)

# Fichiers de mapping pour transformer les fichier CSV en fichier TTL
osm_mapping_file_name = "mapping_osm_adresses.json"
osm_mapping_file = os.path.join(mapping_folder, osm_mapping_file_name)
osm_hn_mapping_file_name = "mapping_osm_hn_adresses.json"
osm_hn_mapping_file = os.path.join(mapping_folder, osm_hn_mapping_file_name)

# Fichiers TTL pour structurer les connaissances venant d'OSM
osm_kg_file_name = "osm_adresses.ttl"
osm_kg_file = os.path.join(tmp_folder, osm_kg_file_name)
osm_hn_kg_file_name = "osm_adresses_hn.ttl"
osm_hn_kg_file = os.path.join(tmp_folder, osm_hn_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
osm_factoids_kg_file_name = "osm_factoids.ttl"
osm_factoids_kg_file = os.path.join(tmp_folder, osm_factoids_kg_file_name)
osm_permanent_kg_file_name = "osm_permanent.ttl"
osm_permanent_kg_file = os.path.join(tmp_folder, osm_permanent_kg_file_name)

osm_time_description = {
    "start_time" : {"stamp":"2024-01-01T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

## Processus final et itératif

### Création des factoïdes dans des répertoires

Pour chaque source, des factoïdes sont créés indépendamment dans des répertoires distincts

#### Ville de Paris


In [ ]:
# fc.create_factoid_process_ville_paris(graphdb_url, vpt_repository_name, namespace_prefixes, tmp_folder,
#                                       ontorefine_url, ontorefine_cmd, ont_file, ontology_named_graph_name, factoids_named_graph_name, permanent_named_graph_name,
#                                       vpta_csv_file, vpta_mod_csv_file, vpta_csv_file_delimiter, vpta_csv_file_geom_col_name,
#                                       vptc_csv_file, vpta_mapping_file, vptc_mapping_file, vpta_kg_file, vptc_kg_file,
#                                       vpta_time_description)

####  BAN


In [ ]:
# fc.create_factoid_process_ban(graphdb_url, bpa_repository_name, namespace_prefixes, tmp_folder,
#                               ontorefine_url, ontorefine_cmd,
#                               ont_file, ontology_named_graph_name, factoids_named_graph_name, permanent_named_graph_name,
#                               bpa_csv_file, bpa_mapping_file, bpa_kg_file,
#                               bpa_time_description)

#### Wikidata


In [ ]:
# # fc.get_data_from_wikidata(wdpt_csv_file, wdpa_csv_file, wdpl_csv_file)
# fc.create_factoid_process_wikidata(graphdb_url, wd_repository_name, namespace_prefixes, tmp_folder,
#                                    ontorefine_url, ontorefine_cmd,
#                                    ont_file, ontology_named_graph_name, factoids_named_graph_name, permanent_named_graph_name,
#                                    wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file,
#                                    wdpa_csv_file, wdpa_mapping_file, wdpa_kg_file,
#                                    wdpl_csv_file, wdpl_mapping_file, wdpl_kg_file)

#### OSM

In [ ]:
# fc.create_factoid_process_osm(graphdb_url, osm_repository_name, namespace_prefixes, tmp_folder,
#                               ontorefine_url, ontorefine_cmd, ont_file, ontology_named_graph_name, factoids_named_graph_name, permanent_named_graph_name,
#                               osm_csv_file, osm_mapping_file, osm_kg_file,
#                               osm_hn_csv_file, osm_hn_mapping_file, osm_hn_kg_file,
#                               osm_time_description)

### Insertion des factoïdes dans le graphe des faits

#### Ville de Paris

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, vpt_repository_name, vpt_factoids_kg_file, vpt_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

#### Wikidata

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, wd_repository_name, wd_factoids_kg_file, wd_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

#### BAN

In [ ]:
# msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, bpa_repository_name, bpa_factoids_kg_file, bpa_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
# msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

#### OSM

In [ ]:
# msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, osm_repository_name, osm_factoids_kg_file, osm_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
# msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

## Intégration de données issues de fichiers Geojson

Ces fichiers sont issus de la vectorisation de plans de Paris
* cadatre napoléonien révisé de 1847
* plan "Andriveau" de 1849
* plan parcellaire municipal de 1871
* plan de l'Atlas Municipal de 1888

### Variables globale pour l'import des données venant des fichiers Geojson

In [ ]:
lang = "fr"
landmark_type = "Thoroughfare"
geojson_join_property = "name"

### Variables liées au cadastre napoléonien de 1847

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
cn_1847_repository_name = "factoids_1847_cadastre_nap"

cn_1847_geojson_file_name = "1847_cadastre_nap.geojson"
cn_1847_geojson_file = os.path.join(data_folder, cn_1847_geojson_file_name)
cn_1847_kg_file_name = "cn_1847_kg.ttl"
cn_1847_kg_file = os.path.join(tmp_folder, cn_1847_kg_file_name)

# Fichier TTL final des factoïdes du cadatre napoléonien révisé de 1847
cn_1847_factoids_kg_file_name = "cn_1847_factoids.ttl"
cn_1847_factoids_kg_file = os.path.join(tmp_folder, cn_1847_factoids_kg_file_name)
cn_1847_permanent_kg_file_name = "cn_1847_permanent.ttl"
cn_1847_permanent_kg_file = os.path.join(tmp_folder, cn_1847_permanent_kg_file_name)

cn_1847_geojson = fm.read_json_file(cn_1847_geojson_file)

cn_1847_source_desc = {
    "lang" : "fr", 
    "label" : "Cadastre napoléonien de Gentilly de 1847",
    "publisher" : {
        "label": "Empire français"
        }
}

cn_1847_time_interval = {
    "start_time" : {"stamp":"1845-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1850-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
cn_1847_geojson["source"] = cn_1847_source_desc
cn_1847_geojson["time"] = cn_1847_time_interval


### Variables liées au plan d'Andriveau

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
an_1849_repository_name = "factoids_1849_andriveau"

an_1849_geojson_file_name = "1849_andriveau.geojson"
an_1849_geojson_file = os.path.join(data_folder, an_1849_geojson_file_name)
an_1849_kg_file_name = "an_1849_kg.ttl"
an_1849_kg_file = os.path.join(tmp_folder, an_1849_kg_file_name)

# Fichier TTL final des factoïdes du plan "Andriveau" de 1849
an_1849_factoids_kg_file_name = "an_1849_factoids.ttl"
an_1849_factoids_kg_file = os.path.join(tmp_folder, an_1849_factoids_kg_file_name)
an_1849_permanent_kg_file_name = "an_1849_permanent.ttl"
an_1849_permanent_kg_file = os.path.join(tmp_folder, an_1849_permanent_kg_file_name)

an_1849_geojson = fm.read_json_file(an_1849_geojson_file)

an_1849_source_desc = {
    "lang" : "fr", 
    "label" : "Plan d'Andriveau de 1849",
    "publisher" : {
        "label": "Andriveau"
        }
}

an_1849_time_interval = {
    "start_time" : {"stamp":"1847-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1851-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
an_1849_geojson["source"] = an_1849_source_desc
an_1849_geojson["time"] = an_1849_time_interval

### Variables liées au plan parcellaire municipal de 1871

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
pm_1871_repository_name = "factoids_1871_plan_parcellaire_mun"

pm_1871_geojson_file_name = "1871_plan_parcellaire_mun.geojson"
pm_1871_geojson_file = os.path.join(data_folder, pm_1871_geojson_file_name)
pm_1871_kg_file_name = "pm_1871_kg.ttl"
pm_1871_kg_file = os.path.join(tmp_folder, pm_1871_kg_file_name)

# Fichier TTL final des factoïdes du plan parcellaire municipal de 1871
pm_1871_factoids_kg_file_name = "pm_1871_factoids.ttl"
pm_1871_factoids_kg_file = os.path.join(tmp_folder, pm_1871_factoids_kg_file_name)
pm_1871_permanent_kg_file_name = "pm_1871_permanent.ttl"
pm_1871_permanent_kg_file = os.path.join(tmp_folder, pm_1871_permanent_kg_file_name)

pm_1871_geojson = fm.read_json_file(pm_1871_geojson_file)

pm_1871_source_desc = {
    "lang" : "fr",
    "label" : "Plan parcellaire municipal",
    "publisher" : {
        "label": "IIIe République"
        }
}

pm_1871_time_interval = {
    "start_time" : {"stamp":"1870-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1872-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
pm_1871_geojson["source"] = pm_1871_source_desc
pm_1871_geojson["time"] = pm_1871_time_interval

### Variables liées au plan municipal de 1888

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
am_1888_repository_name = "factoids_1888_atlas_municipal"

am_1888_geojson_file_name = "1888_atlas_municipal.geojson"
am_1888_geojson_file = os.path.join(data_folder, am_1888_geojson_file_name)
am_1888_kg_file_name = "am_1888_kg.ttl"
am_1888_kg_file = os.path.join(tmp_folder, am_1888_kg_file_name)

# Fichier TTL final des factoïdes du plan de l'Atlas Municipal de 1888
am_1888_factoids_kg_file_name = "am_1888_factoids.ttl"
am_1888_factoids_kg_file = os.path.join(tmp_folder, am_1888_factoids_kg_file_name)
am_1888_permanent_kg_file_name = "am_1888_permanent.ttl"
am_1888_permanent_kg_file = os.path.join(tmp_folder, am_1888_permanent_kg_file_name)

am_1888_geojson = fm.read_json_file(am_1888_geojson_file)

am_1888_source_desc = {
    "lang" : "fr", 
    "label" : "Plan de l'atlas municipal de 1888",
    "publisher" : {
        "label": "Ville de Paris"
        }
}

am_1888_time_interval = {
    "start_time" : {"stamp":"1887-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1889-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
am_1888_geojson["source"] = am_1888_source_desc
am_1888_geojson["time"] = am_1888_time_interval

In [ ]:
# fc.create_factoid_process_geojson(graphdb_url, cn_1847_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
#                                factoids_named_graph_name, permanent_named_graph_name, cn_1847_geojson, geojson_join_property, cn_1847_kg_file, landmark_type, lang)
# fc.create_factoid_process_geojson(graphdb_url, an_1849_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
#                                factoids_named_graph_name, permanent_named_graph_name, an_1849_geojson, geojson_join_property, an_1849_kg_file, landmark_type, lang)
# fc.create_factoid_process_geojson(graphdb_url, pm_1871_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
#                                factoids_named_graph_name, permanent_named_graph_name, pm_1871_geojson, geojson_join_property, pm_1871_kg_file, landmark_type, lang)
# fc.create_factoid_process_geojson(graphdb_url, am_1888_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
#                                factoids_named_graph_name, permanent_named_graph_name, am_1888_geojson, geojson_join_property, am_1888_kg_file, landmark_type, lang)

In [ ]:
# msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, cn_1847_repository_name,
#                                        cn_1847_factoids_kg_file, cn_1847_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
# msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

# msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, an_1849_repository_name,
#                                        an_1849_factoids_kg_file, an_1849_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
# msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

# msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, pm_1871_repository_name,
#                                        pm_1871_factoids_kg_file, pm_1871_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
# msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

# msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, am_1888_repository_name,
#                                        am_1888_factoids_kg_file, am_1888_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
# msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

In [ ]:
def add_time_relations(graphdb_url:str, repository_name:str, time_named_graph_name:str):
    """
    Ajout de relations temporelles :
    * comparaison des instants appartenant à un même événement (i1 before/after i2)
    * comparaison des instants liés à un même attribut (i1 before/after i2)
    * déduction des instants au plus tôt / au plus tard liés aux événéments à partir des instants avant / après
    * création d'intervalles de validité pour des versions d'attributs
    * comparaison des intervalles de versions entre versions d'un même attribut

    L'ensemble des triplets est stocké dans le graphe nommé dont le nom est `time_named_graph_name`.
    """
    
    time_named_graph_uri = URIRef(gd.get_named_graph_uri_from_name(graphdb_url, repository_name, time_named_graph_name))

    tm.compare_time_instants_of_events(graphdb_url, repository_name, time_named_graph_uri)
    tm.compare_time_instants_of_attributes(graphdb_url, repository_name, time_named_graph_uri)
    tm.get_earliest_and_latest_time_instants_for_events(graphdb_url, repository_name, time_named_graph_uri)
    tm.get_validity_interval_for_attribute_versions(graphdb_url, repository_name, time_named_graph_uri)
    tm.compare_time_intervals_of_attribute_versions(graphdb_url, repository_name, time_named_graph_uri)

add_time_relations(graphdb_url, facts_repository_name, time_named_graph_name)

In [ ]:
import pyproj
from shapely.ops import transform
from shapely import wkt
import re

def get_insert_data_query_for_version_comparisons(version_comparisons:list[tuple], named_graph_uri:URIRef=None):
    query_lines = ""
    for comp in version_comparisons:
        if comp[2]:
            pred = "addr:sameVersionValueAs"
        else:
            pred = "addr:differentVersionValueFrom"

        query_lines += f"{comp[0].n3()} {pred} {comp[1].n3()} .\n"

    if named_graph_uri is None:
        opened_named_graph = ""
        closed_named_graph = ""
    else:
        opened_named_graph = f"GRAPH {named_graph_uri.n3()} {{"
        closed_named_graph = f"}}"
        
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>

        INSERT DATA {{
        {opened_named_graph}
        {query_lines}
        {closed_named_graph}
        }} 
        """

    return query

def get_wkt_geom_from_geosparql_wktliteral(wktliteral:str):
    wkt_srid_pattern = "<(.{0,})>"
    wkt_value_pattern = "<.{0,}> {1,}"
    wkt_geom_srid_match = re.match(wkt_srid_pattern, wktliteral)
    
    epsg_4326_uri = URIRef("http://www.opengis.net/def/crs/EPSG/0/4326")
    crs84_uri = URIRef("http://www.opengis.net/def/crs/OGC/1.3/CRS84")

    if wkt_geom_srid_match is not None:
        wkt_geom_srid = URIRef(wkt_geom_srid_match.group(1))
    else:
        wkt_geom_srid = epsg_4326_uri

    if wkt_geom_srid == crs84_uri:
        wkt_geom_srid = epsg_4326_uri
    wkt_geom_value = re.sub(wkt_value_pattern, "", wktliteral)

    return wkt_geom_value, wkt_geom_srid

def transform_geometry_crs(geom, crs_from, crs_to):
    """
    Obtenir une géométrie définie dans le système de coordonnées `from_crs` vers le système `to_crs`.
    """

    project = pyproj.Transformer.from_crs(crs_from, crs_to, always_xy=True).transform
    return transform(project, geom)

def get_pyproj_crs_from_opengis_epsg_uri(opengis_epsg_uri:URIRef):
    pattern = "http://www.opengis.net/def/crs/EPSG/0/([0-9]{1,})"
    try :
        epsg_code = re.match(pattern, opengis_epsg_uri.strip()).group(1)
        return pyproj.CRS(f'EPSG:{epsg_code}')
    except :
        return None

def are_similar_geometries(geom_1, geom_2):
    # geom_intersection = geom_1.intersection(geom_2)
    # geom_union = geom_1.union(geom_2)
    # coef = geom_intersection.area/geom_union.area

    # if coef > 0.7:
    #     return True
    # else:
    #     return False

    geom_intersection = geom_1.envelope.intersection(geom_2.envelope)
    geom_union = geom_1.envelope.union(geom_2.envelope)
    coef = geom_intersection.area/geom_union.area

    if coef >= 0.8:
        print(coef)
        return True
    else:
        return False

def get_processed_geometry(geom_wkt, geom_srid_uri:URIRef, crs_uri:URIRef, buffer_radius:float):
    """
    Obtention d'une géométrie pour pouvoir la comparer aux autres :
    * ses coordonnées seront exprimées dans le référentiel lié à `crs_uri`
    * si la géométrie est une ligne ou un point (area=0.0), alors on récupère une zone tampon dont le buffer est donné par `buffer_radius`
    """

    geom = wkt.loads(geom_wkt)
    crs_from = get_pyproj_crs_from_opengis_epsg_uri(geom_srid_uri)
    crs_to = get_pyproj_crs_from_opengis_epsg_uri(crs_uri)

    # Conversion de la géométrie vers le système de coordonnées cible
    if crs_from != crs_to:
        geom = transform_geometry_crs(geom, crs_from, crs_to)
    
    # Ajout d'un buffer `meter_buffer` mètres si c'est pas un polygone
    if geom.area == 0.0:
        geom = geom.buffer(buffer_radius)

    return geom

def get_geometry_versions(graphdb_url:str, repository_name:str, crs_uri:URIRef, buffer_radius:float):
    query = """
        PREFIX atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/>
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>

        SELECT ?attr ?av ?geom WHERE {
            ?attr a addr:Attribute ; addr:isAttributeType atype:Geometry ; addr:hasAttributeVersion ?av.
            ?av addr:versionValue ?geom.
        }
        """
    
    a = gd.select_query_to_json(query, graphdb_url, repository_name)
    geom_versions = {}

    for elem in a.get("results").get("bindings"):

        # Récupération des URIs (attibut et version d'attribut) et de la géométrie
        rel_attr = gr.convert_result_elem_to_rdflib_elem(elem.get('attr'))
        rel_av = gr.convert_result_elem_to_rdflib_elem(elem.get('av'))
        rel_geom = gr.convert_result_elem_to_rdflib_elem(elem.get('geom'))

        if rel_attr not in geom_versions.keys():
            geom_versions[rel_attr] = {}

        geom_wkt, geom_srid_uri = get_wkt_geom_from_geosparql_wktliteral(rel_geom.strip())
        geom = get_processed_geometry(geom_wkt, geom_srid_uri, crs_uri, buffer_radius)
        geom_versions[rel_attr][rel_av] = geom

    return geom_versions

def test_geom(graphdb_url:str, repository_name:str, comp_named_graph_name:str, crs_uri:URIRef, buffer_radius:float):
    
    geom_versions = get_geometry_versions(graphdb_url, repository_name, crs_uri, buffer_radius)
    version_comparisons = []
    for attr_vers_uris in geom_versions.values():
        for attr_vers_uri_1, geom_1 in attr_vers_uris.items():
            for attr_vers_uri_2, geom_2 in attr_vers_uris.items():
                if attr_vers_uri_1 != attr_vers_uri_2:
                    sim_geoms = are_similar_geometries(geom_1, geom_2)
                    version_comparisons.append((attr_vers_uri_1, attr_vers_uri_2, sim_geoms))

    comp_named_graph_uri = URIRef(gd.get_named_graph_uri_from_name(graphdb_url, repository_name, comp_named_graph_name))
    query = get_insert_data_query_for_version_comparisons(version_comparisons, comp_named_graph_uri)
    # print(query)
    # gd.update_query(query, graphdb_url, repository_name)

In [ ]:
crs_uri = URIRef('http://www.opengis.net/def/crs/EPSG/0/2154')
comp_named_graph_name = "version_comparisons"
test_geom(graphdb_url, facts_repository_name, comp_named_graph_name, crs_uri, buffer_radius=5)

In [ ]:
# wkt_geom_1 = "GEOMETRYCOLLECTION (MULTILINESTRING ((652389.475687 6858949.618137, 652473.25595 6859042.910018)), MULTILINESTRING ((652473.25595 6859042.910018, 652487.397191 6859059.790447)), MULTILINESTRING ((652543.71011 6859097.31558, 652622.978439 6859129.647989)), MULTILINESTRING ((652488.514709 6859073.780449, 652543.71011 6859097.31558)), MULTILINESTRING ((652370.309844 6858929.780064, 652389.475687 6858949.618137)))"
# wkt_geom_2 = "POLYGON ((2.352701 48.829847, 2.352732 48.829719, 2.352699 48.829732, 2.352572 48.829637, 2.352545 48.829617, 2.352518 48.829596, 2.352487 48.829573, 2.352349 48.829469, 2.352128 48.829304, 2.352087 48.829273, 2.352049 48.829245, 2.352064 48.829237, 2.352006 48.829194, 2.351976 48.829172, 2.351955 48.829156, 2.351925 48.829134, 2.351868 48.829093, 2.351853 48.829081, 2.35177 48.82902, 2.351768 48.829021, 2.351757 48.829027, 2.351706 48.828989, 2.351682 48.82897, 2.351661 48.828955, 2.351637 48.828937, 2.351632 48.828933, 2.351641 48.828928, 2.351625 48.828917, 2.351599 48.828896, 2.351417 48.828752, 2.351317 48.828677, 2.351224 48.82868, 2.351223 48.828726, 2.35135 48.82882, 2.351497 48.828931, 2.351591 48.829001, 2.351605 48.829011, 2.351684 48.829071, 2.35173 48.829105, 2.351881 48.829218, 2.351864 48.829227, 2.352004 48.829332, 2.352022 48.829345, 2.352076 48.829386, 2.35209 48.829397, 2.35218 48.829464, 2.352197 48.829454, 2.352221 48.829472, 2.352238 48.829485, 2.352252 48.829495, 2.352283 48.829518, 2.352361 48.829577, 2.352381 48.829591, 2.352429 48.829627, 2.352507 48.829686, 2.352538 48.829666, 2.352557 48.82968, 2.352593 48.829708, 2.352615 48.829724, 2.352634 48.829738, 2.352641 48.829786, 2.352686 48.82982, 2.352701 48.829847))"
# geom_1 = wkt.loads(wkt_geom_1)
# geom_2 = wkt.loads(wkt_geom_2)

# bbox_geom_1 = geom_1.envelope
# bbox_geom_2 = geom_2.envelope
# print(bbox_geom_1)
